<a href="https://colab.research.google.com/github/b62mp1/files/blob/main/p3fc1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
pip install yfinance

In [119]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Activation, concatenate, Dropout
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go


In [139]:
stock_name = "TSLA"
dataset=pdr.get_data_yahoo(stock_name, start='2020-01-01', end='2022-04-19').reset_index()
dataset.index = dataset['Date']

print("Number of rows and columns:", dataset.shape)
dataset.head(5)

[*********************100%***********************]  1 of 1 completed
Number of rows and columns: (578, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2020-01-02,2020-01-02,84.900002,86.139999,84.342003,86.052002,86.052002,47660500
2020-01-03,2020-01-03,88.099998,90.800003,87.384003,88.601997,88.601997,88892500
2020-01-06,2020-01-06,88.094002,90.311996,88.000000,90.307999,90.307999,50665000
2020-01-07,2020-01-07,92.279999,94.325996,90.671997,93.811996,93.811996,89410500
2020-01-08,2020-01-08,94.739998,99.697998,93.646004,98.428001,98.428001,155721500


In [140]:
close_data = dataset['Close'].values
close_data = close_data.reshape((-1,1))

scaler = MinMaxScaler(feature_range=(0, 1))
close_data = scaler.fit_transform(close_data)

In [141]:

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = dataset['Date'][:split]
date_test = dataset['Date'][split:]


In [142]:
look_back = 5

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=10)

In [143]:
model = Sequential()
model.add(
    LSTM(1000,
        activation='relu',
        input_shape=(look_back,1)
        )
)

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit(train_generator,epochs=num_epochs, verbose=1)

Epoch 1/25
23/23 [==============================] - 4s 131ms/step - loss: 0.0471
Epoch 2/25
23/23 [==============================] - 3s 132ms/step - loss: 0.0062
Epoch 3/25
23/23 [==============================] - 3s 131ms/step - loss: 0.0021
Epoch 4/25
23/23 [==============================] - 3s 131ms/step - loss: 0.0012
Epoch 5/25
23/23 [==============================] - 3s 131ms/step - loss: 9.3821e-04
Epoch 6/25
23/23 [==============================] - 3s 129ms/step - loss: 9.7278e-04
Epoch 7/25
23/23 [==============================] - 3s 129ms/step - loss: 0.0018
Epoch 8/25
23/23 [==============================] - 3s 131ms/step - loss: 0.0013
Epoch 9/25
23/23 [==============================] - 3s 131ms/step - loss: 0.0013
Epoch 10/25
23/23 [==============================] - 3s 130ms/step - loss: 0.0014
Epoch 11/25
23/23 [==============================] - 3s 130ms/step - loss: 9.7351e-04
Epoch 12/25
23/23 [==============================] - 3s 129ms/step - loss: 0.0015
Epoch 13/25
2

In [144]:
prediction = model.predict(test_generator)

In [145]:

def predict(num_prediction, model):
    prediction_list = close_test[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = dataset['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [146]:
#close_test=close_test[:-look_back]

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data',
)


trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Predicted Data'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Test Data'
)
trace4 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode='lines',
    name = 'Future Data'
)
layout = go.Layout(
    title = stock_name,
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
#fig.update_xaxes(type="log")

fig.show()

In [147]:
close_test2=close_test[:-5]
RNNmse = mean_squared_error(close_test2,prediction)
from math import sqrt
RNNrmse = sqrt(RNNmse)
RNNr2Score = r2_score(close_test2,prediction)

RNNmse,RNNrmse,RNNr2Score

(0.0018431988017370087, 0.042932491212798365, 0.8099498779561275)